# get menu from the locu api

In [ ]:
# read from the previously generated menu file 
def readBuzs():
    if os.path.isfile(BUZ_PICKLE_FILE):
        buzs = pickle.load(open(BUZ_PICKLE_FILE, 'rb'))
    else:
        buzs = {}
    if os.path.isfile(BUZ_MENU_PICKLE_FILE):
        buzs_with_menu = pickle.load(open(BUZ_MENU_PICKLE_FILE, 'rb'))
    else:
        buzs_with_menu = {}
    return (buzs, buzs_with_menu)

In [ ]:
import json, os
import pickle
import requests

BUZ_PICKLE_FILE = 'buzs.pickle'
BUZ_MENU_PICKLE_FILE = 'buzs_with_menu.pickle'

buzs, buzs_with_menu = readBuzs()
count = 0
with open("business.json") as f:
    for line in f:
        buz = json.loads(line)
        count += 1
        print(count)
        # Skip if this venue is not a restaurant 
        if "Restaurants" not in buz["categories"]:
            continue
        # Skip if this restaurant has been read 
        if buz["business_id"] in buzs:
            continue
        # Dictionary that map "business_id" to corresponding restaurant
        buzs[buz["business_id"]] = buz
        url = "https://api.locu.com/v2/venue/search"
        params = {
            "fields" : [ "name", "menus" ],
            "venue_queries" : [
                {
                    "name" : buz["name"],
                    "location" : {"region" : buz["state"], "locality" : buz["city"], "postal_code" : buz["postal_code"]},
                    "menus" : { "$present" : True },
                }
            ],
            "api_key" : "af31648d127d362397e9ad94c8df49665111f52c",
        }
        # Equals to data=json.dumps(params)
        response = requests.post(url, json=params)
        if response.status_code != 200:
            print('Status code error:', response.status_code)
            #print(response.text)
            del buzs[buz['business_id']]
            continue
        content = json.loads(response.content)
        if len(content["venues"]) == 0:
            continue
        print("New buz_with_menu count:", count)
        buz['venues'] = content['venues']
        buzs_with_menu[buz['business_id']] = buz


pickle.dump(buzs, open('buzs.pickle', 'wb'))
pickle.dump(buzs_with_menu, open('buzs_with_menu.pickle', 'wb'))
print('Final result')
print('Num of buzs:', len(buzs))
print('Num of buzs with menu:', len(buzs_with_menu))

# del venue in buzs, append menu

In [ ]:
'''
Extract menu info from raw data.
After running this script, buzs_simple.pickle should contain a dictionary
of buz_id -> buz, where buz has basic business info as well as menu data in
buz['menu']. buz['menu'] contains a list of food.
Input file: buzs_with_menu.pickle
Output file: buzs_simple.pickle
'''

buzs = pickle.load(open('buzs_with_menu.pickle', 'rb'))
buzs_simple = {}

count = 0
for buz_id, buz in buzs.items():
    count += 1
    #print(count, buz_id)
    if ('venues' not in buz) or (len(buz['venues']) == 0):
        print('No venues for {}'.format(buz_id))
    venue = buz['venues'][0]
    if ('menus' not in venue) or (len(venue['menus']) == 0):
        print('No menus for {}'.format(buz_id))
    menus = venue['menus']
    food = []
    for menu in menus:
        for section in menu['sections']:
            for subsection in section['subsections']:
                for content in subsection['contents']:
                    count1 += 1
                    food.append(content)
    del buz['venues']
    buz['menu'] = food
    buzs_simple[buz_id] = buz
pickle.dump(buzs_simple, open('buzs_simple.pickle', 'wb'))

# tokenize review

In [417]:
from nltk import word_tokenize, sent_tokenize
'''
Tokenize business reviews. Each review becomes a list of tokens. Sentences are separated
by '\n' token.
Input file: yelp_academic_dataset_review.json
            buzs_simple.pickle
Output file: tokenized_reviews.pickle
'''

REVIEW_FILE_NAME = 'review.json'
BUZ_FILE_NAME = 'buzs_simple.pickle'

buzs = pickle.load(open(BUZ_FILE_NAME, 'rb'))
buz_ids = buzs.keys()

buz_reviews = {buz_id: [] for buz_id in buz_ids}

# tokenize reviews
total_count = 0
relevant_count = 0
with open(REVIEW_FILE_NAME) as f:
    for line in f:
        if(total_count) > 1000:
            break
        total_count += 1
        review = json.loads(line)
        buz_id = review['business_id']
        if buz_id in buz_ids:
            relevant_count += 1
            # Remove punctuation and Foreign characters
            review_text = review['text']
            print(review_text)
            review_text = unicodedata.normalize('NFD', review_text).encode('ascii', 'ignore').decode('utf-8')
            sent_tokens = sent_tokenize(review_text)
            sents = []
            for sent in sent_tokens:
                sent = re.sub(r'(w/)', 'with', sent)
                sent = re.sub(r'[\d]|[^\w\s]', '', sent)
                tokens = word_tokenize(sent)
                tokens = [token.lower() for token in tokens if len(token) < 20]
                if len(tokens) > 50:
                    continue
                sents.append(tokens)
            review['text'] = sents
            buz_reviews[buz_id].append(review)
        if total_count % 10000 == 0:
            print('Total review processed:', total_count)
            print('Relevant review:', relevant_count)

pickle.dump(buz_reviews, open('tokenized_reviews.pickle', 'wb'))

I left Table 17 feeling very ambivalent. Meh as others would put it. Some things good somethings not so great but on the whole in between.

The room is simple, farmhouse chic: long harvest table, ornamental roosters and wooden chairs with cushions for seating. Sundays and Mondays they have a decent prix fix menu on offer or you can order a la carte from their shared plates menu. I went with the steak frites and salad off the table d'hote. Spinach salad came nicely dressed though the dressing was plain could have used more tartness. I had a few slight issues with the steak frites, the steak came closer to medium when medium rare was requested and was under seasoned. My accompaniment of frites (which were wonderful btw) on the other hand was too liberally salted with kosher salt so I guess I got the best of both worlds! The Bearnaise sauce sauce on the side was so-so and didn't do too much to perk up the steak. Dessert was chilled rhubarb with creme anglaise really lovely but too much su

Disappointing experience.  The restaurant was packed and my friend and I were crammed into a small table where we were basically touching elbows with the tables beside us.  The food wasn't anything special which surprised me - I had been intrigued to try this place for a while but I won't be going back.
Went for Sunday brunch. Nacoise salad for me, scramble of the day for my friend.  Waiter had an attitude. Food was good / acceptable. Decor was lovely (especially the bathroom!), however  I wouldn't go back again.
Table 17 is a quaint find in Leslieville village perfect for a good catch up with your girlfriends or a dinner date.  The restaurant is divided into two rooms - the first room you walk into features a long bar and some seating and the room towards the side which we get seated in, is simple, farmhouse chic.  The menu has amazing sharing plates and we do choose a couple to start with. 
We were very impressed with the presentation of the polenta du jour dish.  This is a must at T

I love this location! The staff is very friendly and helpful. Also, you just can't beat the $5 meal special.


# tokenize menu

In [332]:
import nltk
import pickle, re
import string 
import unicodedata

'''
Tokenize menu data.
Input file: buzs_simple.pickle
Output file: buzs_tokenized.pickle
    contains a dictionary from buz_id -> business.
'''

BUZ_FILE_NAME = 'buzs_simple.pickle'
TOKENIZED_FILE_NAME = 'buzs_tokenized.pickle'

buzs = pickle.load(open(BUZ_FILE_NAME, 'rb'))

count = 0
for buz_id, buz in buzs.items():
    count += 1
    menu = []
    for food in buz['menu']:
        if food['type'] != 'ITEM' or 'name' not in food:
            continue
        food['name'] = re.sub(r'(w/)',"with",food['name'])
        food['name'] = re.sub(r'[\d]|(\([^)]*\))|[^\w\s]', '', food['name'])
        food['name'] = unicodedata.normalize('NFD', food['name']).encode('ascii', 'ignore').decode('utf-8')
        sent_tokens = sent_tokenize(food['name'])
        tokens = word_tokenize(food['name'])
        tokens = [token.lower() for token in tokens]
        food['name'] = tokens
        menu.append(food)
    buz['menu'] = menu
    if count % 50 == 0:
        print('count:', count)
        print(buz['menu'][0])

pickle.dump(buzs, open(TOKENIZED_FILE_NAME, 'wb'))

count: 50
{'description': 'seared chicken & vegetable dumplings, asian slaw, traditional soy dipping sauce', 'name': ['potstickers'], 'price': '10.25', 'type': 'ITEM'}
count: 100
{'type': 'ITEM', 'description': 'Mixed greens, tomato, red onion, cucumber, sweet bell pepper and alfalfa sprouts tossed and garnished with strips of fresh mango and mint served with our delicious homemade ginger dressing', 'name': ['bamboo', 'legend', 'salad'], 'price': '7.99', 'option_groups': [{'text': 'Meat', 'options': [{'name': 'Chicken', 'price': '+2.00'}, {'name': 'Shrimp', 'price': '+3.00'}], 'type': 'OPTION_ADD'}]}
count: 150
{'description': 'Prosciutto | basil | marinara', 'name': ['fried', 'stuffed', 'provolone'], 'price': '7.00', 'type': 'ITEM'}
count: 200
{'description': 'A third-pound* 100% Angus beef patty with its own special seasoning on a premium bakery style bun, hickory smoked bacon, two slices of melty American cheese and topped off with crisp red onions and Crinkle-cut pickles.', 'name':

count: 2100
{'price': '8.00', 'type': 'ITEM', 'name': ['espresso', 'love'], 'description': 'Patrón XO cafe, espresso & dark chocolate. Like a great coffeehouse experience while hanging out in a bar, which is way more fun.'}
count: 2150
{'price': '6.25', 'type': 'ITEM', 'name': ['stuffed', 'banana', 'peppers'], 'description': 'With bread and butter'}
count: 2200
{'price': '1.50', 'type': 'ITEM', 'name': ['miso', 'shiru'], 'description': 'soy bean soup'}
count: 2250
{'price': '4.99', 'type': 'ITEM', 'name': ['eggs', 'lots', 'of', 'homefries', 'double', 'texas', 'toast']}
count: 2300
{'type': 'ITEM', 'name': ['whopper', 'sandwich', 'meal'], 'description': 'Our WHOPPER® Sandwich Meal is a ¼ lb. of savory flame-broiled beef topped with juicy tomatoes, fresh cut lettuce, creamy mayonnaise, crunchy pickles, and sliced white onions on a soft sesame seed bun. Served with a small side of piping hot, thick cut French Fries or golden Onion Rings and a small fountain drink of your choice to make it

# match menu and food entity in review, label them using 'BILOU'

In [ ]:
import json, os
import pickle
import requests

In [461]:
def is_match(start_idx, review_tokens, food_tokens):
    matched_count = len(food_tokens)
    if start_idx + len(food_tokens) > len(review_tokens):
        return False
    if review_tokens[start_idx] != food_tokens[0].lower():
        return False
    for idx in range(len(food_tokens)):
        if review_tokens[start_idx + idx].lower() != food_tokens[idx].lower():
            matched_count = matched_count - 1
            if matched_count/len(food_tokens) < 2/3:
                return False
    return True

In [458]:
REVIEW_FILE_NAME = 'tokenized_reviews.pickle'
BUZ_FILE_NAME = 'buzs_tokenized.pickle'
REVIEW_LABEL_FILE_NAME = 'labeled_reviews.txt'
buz_reviews = pickle.load(open(REVIEW_FILE_NAME, 'rb'))
buzs = pickle.load(open(BUZ_FILE_NAME, 'rb'))

In [462]:
review_count = 0
matched_sent_count = 0
tokenized_reviews = []
with open(REVIEW_LABEL_FILE_NAME, 'w') as f:
    for buz_id, reviews in buz_reviews.items():
        menu = buzs[buz_id]['menu']
        for review in reviews:
            review_count += 1
            if review_count % 1000 == 0:
                print('review count:', review_count)
                print('sentence count:', matched_sent_count)
            for tokens in review['text']: 
                labeled_tokens = []
                if len(tokens) == 0:
                    continue
                idx = 0
                matched_food_count = 0
                total_food_length=0
                while idx < len(tokens):
                    # check match
                    match = False
                    for food in menu:
                        food_tokens = food['name']
                        if len(food_tokens) < 1:
                            continue
                        match = is_match(idx, tokens, food_tokens)
                        if match:
                            matched_food_count += 1
                            food_len = len(food_tokens)
                            matched_len = (food_len-1, food_len)[tokens[idx+food_len-1]==food_tokens[-1]]
                            if food_len == 1:
                                labeled_tokens.append((tokens[idx], 'U'))
                            else:
                                labeled_tokens.append((tokens[idx], 'B'))
                                for i in range(1, matched_len - 1):
                                    labeled_tokens.append((tokens[idx + i], 'I'))
                                labeled_tokens.append((tokens[idx + matched_len - 1], 'L'))
                            idx += matched_len
                            total_food_length += matched_len
                            break
                    if not match:
                        labeled_tokens.append((tokens[idx], 'O'))
                        idx += 1
                
                if total_food_length/len(tokens)>=0.4 or matched_food_count >= 3:
                    matched_sent_count += 1
                    temp=[]
                    for token in labeled_tokens:
                        temp.append(token[0])
                        f.write(token[0]+'\t'+token[1]+'\n')
                    f.write('\n')
                    tokenized_reviews.append(temp)
    
print('final review count:', review_count)
print('final matched sentence count:', matched_sent_count)       

final review count: 100
final matched sentence count: 12


# Take the labeled_review json file as input, generate txt file (deprecated)

In [120]:
REVIEW_LABEL_TXT_NAME = 'labeled_reviews.txt'
review_count = 0

with open(REVIEW_LABEL_FILE_NAME, 'r') as json_data, open(REVIEW_LABEL_TXT_NAME, 'w') as txt_data:
    for line in json_data:
        review_count += 1
        if review_count % 1000 == 0:
            print('processed reviews: ', review_count)
        review = json.loads(line)
        tokens = review['text']
        for token in tokens:
            txt_data.write(token[0]+'\t'+token[1]+'\n')
        txt_data.write('\n')
    print('final count ', review_count)

final count  501
